In [ ]:
import requests
import pandas as pd
import os,errno
import sys
import time
import numpy as np
import xarray as xr
import datetime as dt
import pandas as pd
import glob

dir_data='/Users/ahenny/'
dir2='/Volumes/Extreme Pro/'

In [ ]:
from urllib.request import urlretrieve
strs_list=['20170530','20170814','20180109','20180912','20171211','20170823','20171024','20171108','20171211',
           '20180108','20180212','20180307','20180409','20180508','20180606','20180713','20180814',
           '20180912','20181010','20181106','20181207','20190115','20190208','20190306','20190407',
           '20190608','20190711','20190807','20190911','20191008','20191106','20191206','20200110',
           '20200206','20200309','20200408','20200509','20200615','20200707','20200806','20200922',
           '20201007','20201109','20201217','20210113','20210226','20210310','20210408','20210510',
           '20230516','20210711','20210810','20211207','20220107','20220208','20230517','20220808',
           '20220907','20221006','20221209','20230106','20230208','20230307']

month_days=[31,28,31,30,31,30,31,31,30,31,30,31]
yrs=np.arange(2010,2023,1)
for i in range(len(yrs)):
    year=yrs[i]
    month_days[1]=28
    if year%4==0:
        month_days[1]=29
    start=0
    if year==1996:
        start=9
    for j in range(start,12):
        if j<9:
            month_str='0'+str(j+1)
        else:
            month_str=str(j+1)
        for k in range(month_days[j]):
            if k<9:
                day_str='0'+str(k+1)
            else:
                day_str=str(k+1)
            
            filename='/Users/ahenny/holding/gpcp_'+str(year)+month_str+day_str+'.nc'
            print(filename)
            
            status='fail'
            for n in strs_list:
                if status=='fail':
                    try:
                        urlretrieve('https://www.ncei.noaa.gov/data/global-precipitation-climatology-project-gpcp-daily/access/'+str(year)+'/gpcp_v01r03_daily_d'+str(year)+month_str+day_str+'_c'+n+'.nc',filename)
                        status='success'
                    except:
                        pass

In [ ]:
month_days=[31,28,31,30,31,30,31,31,30,31,30,31]
yrs=np.arange(1996,2023,1)
for i in range(len(yrs)):
    year=yrs[i]
    month_days[1]=28
    if year%4==0:
        month_days[1]=29
    start=0
    if year==1996:
        start=9
    for j in range(start,12):
        if j<9:
            month_str='0'+str(j+1)
        else:
            month_str=str(j+1)
        for k in range(month_days[j]):
            if k<9:
                day_str='0'+str(k+1)
            else:
                day_str=str(k+1)
            
            filename='/Users/ahenny/holding/gpcp_'+str(year)+month_str+day_str+'.nc'
            print(filename)
            ds=xr.open_dataset(filename)
            precip=ds['precip']
            
            lons_east=[x for x in precip.longitude.values if x>=180.]
            lons_west=[x for x in precip.longitude.values if x<180.]
            p_east=precip.sel(longitude=lons_east)
            p_west=precip.sel(longitude=lons_west)
            p_east['longitude']=[x-360. for x in lons_east]
            precip=xr.concat([p_east,p_west],dim='longitude')
            p_east.close()
            p_west.close()
            
            if j==start and k==0:
                precip_concat=precip
            else:
                precip_concat=xr.concat([precip_concat,precip],dim='time')
                
    dk=xr.Dataset()
    dk['precip']=(('time','lat','lon'),precip_concat.values)
    dk.coords['time']=precip_concat.time
    dk.coords['lat']=precip_concat.latitude.values
    dk.coords['lon']=precip_concat.longitude.values
    dk['precip'].attrs["units"]='mm*day-1'
    dk['lat'].attrs["units"]='degrees_north'
    dk['lon'].attrs["units"]='degrees_east'
    try:
        os.remove(dir_data+'holding/'+'gpcp_'+str(year)+'.nc')
    except OSError:
        pass
    dk.to_netcdf(dir_data+'holding/'+'gpcp_'+str(year)+'.nc')